In [11]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [12]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns = boston_dataset.feature_names)
features = data.drop(['INDUS','AGE'],axis=1)
# features
data
log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices,columns=['price'])


In [13]:
CRIM_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8
property_stats = np.ndarray(shape=(1,11))
# property_stats [0] [CRIM_IDX] = features['CRIM'].mean()
# property_stats [0] [ZN_IDX] = features['ZN'].mean()
# property_stats [0] [CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1,11)

property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [14]:
type(features.mean())

pandas.core.series.Series

In [15]:
type(features.mean().values)

numpy.ndarray

In [16]:
features.mean().values.shape

(11,)

In [17]:
features.mean().values.reshape(1,11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [18]:
regr = LinearRegression().fit(features,target)
fitted_vals = regr.predict(features)

# Challenge : Calculate the MSE and RMSE Using sklearn 
MSE = mean_squared_error(target,fitted_vals)
RMSE = np.sqrt(MSE)

    

In [19]:
def get_log_estimate(nr_rooms,
                    student_per_classroom,
                    next_to_river=False,
                    high_confidence=True) : 
#     Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = student_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    
    else:
        property_stats[0][CHAS_IDX] = 0
    
#     Make Prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
#     calc Range
    if high_confidence:
        upper_bound = log_estimate + 2 * RMSE
        lower_bound = log_estimate - 2 * RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 69

    return log_estimate, upper_bound, lower_bound, interval

In [20]:
get_log_estimate(3, 20, next_to_river = True, high_confidence = False)

(2.776758191480399, 2.9642703266775294, 2.589246056283269, 69)

In [21]:
np.median(boston_dataset.target)

21.2

In [34]:
# challenge : write thr python code that converts the log prices estimates using 1970s prices as well
# as the upper and the lower bounds to today's prices
# round the values o the nearest 1000 dollers
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est,upper,lower,conf = get_log_estimate(9, student_per_classroom=15,
                                            next_to_river = False, high_confidence = False)

# convert to today's dollors
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousands

rounded_est = np.around(dollar_est,-3)
rounded_hi = np.around(dollar_hi,-3)
rounded_low = np.around(dollar_low,-3)

print(f'The Estimated property values is {rounded_est}')
print(f'AT {conf}% confidance the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end')

The Estimated property values is 827000.0
AT 69% confidance the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end


In [43]:
def get_dollar_estimate(rm,ptratio,chas = False, large_range= True) : 
    
    """ Estimate the price of a property in Boston
    
    Keyword Argument
        rm -- number of room in the property
        ptratio -- number of student per teacher in the classroom for the school in the area
        chas -- If the property is next to the river otherwise false
        large_range -- True for a 95% prediction interval, False for a 68% interval
    """
    
    
    if rm<1 or ptratio<1 : 
        print('This is Unrealistic. Try Again')
        return
    
    ZILLOW_MEDIAN_PRICE = 583.3
    SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

    log_est,upper,lower,conf = get_log_estimate(rm, student_per_classroom=ptratio,
                                                next_to_river = chas, high_confidence = large_range)

    # convert to today's dollors
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousands

    rounded_est = np.around(dollar_est,-3)
    rounded_hi = np.around(dollar_hi,-3)
    rounded_low = np.around(dollar_low,-3)

    print(f'The Estimated property values is {rounded_est}')
    print(f'AT {conf}% confidance the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end')

In [42]:
get_dollar_estimate(rm=6,ptratio=30,chas=True)

The Estimated property values is 399000.0
AT 95% confidance the valuation range is
USD 274000.0 at the lower end to USD 581000.0 at the high end


In [44]:
import boston_valuation as val
val.get_dollar_estimate(6,12,True)

The Estimated property values is 783000.0
AT 95% confidance the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end
